In [ ]:
%load_ext autoreload
import os
import sys

cwd = os.getcwd()
sys.path.append(os.path.dirname(cwd))

import numpy as np
import matplotlib.pyplot as plt

from zcu_tools import load_data

server_ip = "100.105.52.95"
server_port = 4999

# Onetone

## Lookback

In [ ]:
# signals, Ts, _ = load_data('data/res/lookback.hdf5')
signals, Ts, _ = load_data('data/res/lookback_q5_i_fid.hdf5', server_ip, server_port)
amps = np.abs(signals)
g_amps, e_amps = amps[:, 0], amps[:, 1]
Ts = Ts * 1e6

plt.figure()
plt.plot(Ts, g_amps, label='ground')
plt.plot(Ts, e_amps, label='excited')
# plt.axvline(0.51432, color='r', linestyle='--', label='trig_offset')
# plt.axvline(0.51432 + 1.0, color='g', linestyle='--', label='readout end')
plt.axvline(0.757-0.4, color='r', linestyle='--', label='trig_offset')
plt.axvline(0.757+2.079-0.4, color='g', linestyle='--', label='readout end')
plt.xlabel('Time (us)')
plt.ylabel('Amplitude (a.u.)')
plt.legend()
plt.show()

## Freq

In [ ]:
signals, fpts, _ = load_data('data/res/res_freq_r5_g300.hdf5')
amps = np.abs(signals)
fpts = fpts / 1e9

# ratio1, ratio2 = 0.47, 0.56
# num1 = int(ratio1 * len(fpts))
# num2 = int(ratio2 * len(fpts))
# fpts = fpts[num1:num2]
# amps = amps[num1:num2]

plt.figure()
plt.plot(fpts, amps, '-o', markersize=1)
plt.xlabel('Frequency (GHz)')
plt.ylabel('Amplitude (a.u.)')
plt.show()

## Power dependence

In [ ]:
signals, fpts, pdrs = load_data("data/res/res_pdr_dep_r5.hdf5")
amps = np.abs(signals).T
amps = amps - np.median(amps, axis=1, keepdims=True)
amps = amps / (np.max(amps, axis=1, keepdims=True) - np.min(amps, axis=1, keepdims=True))
fpts = fpts / 1e9

plt.figure()
plt.imshow(amps, aspect="auto", origin="lower", extent=[fpts[0], fpts[-1], pdrs[0], pdrs[-1]])
plt.axhline(900, color='r', linestyle='--', label='gain=900')
plt.axvline(6.02557, color='g', linestyle='--', label='freq=6025.6MHz')
plt.axvline(6.02822, color='g', linestyle='--', label='freq=6028.2MHz')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Gain (a.u.)')
plt.legend(loc=2)
plt.show()

## Flux dependence

In [ ]:
signals, fpts, flxs = load_data("data/res/res_flux_dep_2.hdf5")
amps = np.abs(signals)
fpts = fpts / 1e9

plt.figure()
plt.imshow(amps, aspect='auto', origin='lower', extent=(flxs[0], flxs[-1], fpts[0], fpts[-1]))
plt.ylabel('Frequency (GHz)')
plt.xlabel('Flux (mA)')
plt.ylim(6.00, 6.05)
plt.show()

## power dependence

In [ ]:
signals, fpts, pdrs = load_data("data/res/res_pdr_dep_r5.hdf5")
amps = np.abs(signals).T
amps = amps - np.median(amps, axis=1, keepdims=True)
amps = amps / (np.max(amps, axis=1, keepdims=True) - np.min(amps, axis=1, keepdims=True))
fpts = fpts / 1e9


plt.figure()
plt.imshow(amps, aspect="auto", origin="lower", extent=[fpts[0], fpts[-1], pdrs[0], pdrs[-1]])
plt.axhline(900, color='r', linestyle='--')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Pump power (a.u.)')
plt.show()

# Twotone

## flux dependence

## Freq

In [ ]:
# signals, fpts, _ = load_data("data/qub/qub_freq_q5_i_rf.hdf5")
# signals, fpts, _ = load_data("data/qub/qub_freq_q5_i_fid.hdf5")
signals, fpts, _ = load_data("2024/12/Data_1230/qub_freq_q5_i_9.hdf5", server_ip)
fpts += 6881.280 * 1e6
amps = np.abs(signals)
fpts = fpts / 1e9

plt.figure()
plt.plot(fpts, amps, '-o', markersize=3)
plt.axvline(7.44693, color='r', linestyle='--', label='freq = 7446.93 MHz')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Amplitude (a.u.)')
plt.legend()
plt.show()

## power dependence

In [ ]:
# signals, fpts, pdrs = load_data("data/qub/qub_pdr_dep_q5_i_rf.hdf5")
signals, fpts, pdrs = load_data("data/qub/qub_pdr_dep_q5_i_fid.hdf5")
amps = np.abs(signals).T
amps = amps - np.median(amps, axis=1, keepdims=True)
fpts = fpts / 1e9

plt.figure()
plt.imshow(amps, aspect="auto", origin="lower", extent=[fpts[0], fpts[-1], pdrs[0], pdrs[-1]])
plt.xlabel('Frequency (GHz)')
plt.ylabel('Pump power (a.u.)')
plt.show()

# Time domain

In [4]:
import zcu_tools.analysis.fitting as ft

def rabi_analyze(
    x: int, signals: float, plot_fit=True, decay=False, max_contrast=False
):
    """
    x: 1D array, sweep points
    signals: 1D array, signal points
    """
    if max_contrast:
        y, _ = convert2max_contrast(signals.real, signals.imag)
    else:
        y = np.abs(signals)

    if decay:
        fit_func = ft.fitdecaysin
        sin_func = ft.decaysin
    else:
        fit_func = ft.fitsin
        sin_func = ft.sinfunc

    pOpt, _ = fit_func(x, y)
    curve = sin_func(x, *pOpt)

    freq = pOpt[2]
    phase = pOpt[3] % 360 - 180
    if phase < 0:
        pi_x = (0.25 - phase / 360) / freq
        pi2_x = -phase / 360 / freq
    else:
        pi_x = (0.75 - phase / 360) / freq
        pi2_x = (0.5 - phase / 360) / freq

    plt.figure()
    plt.plot(x, y, label="meas", ls="-", marker="o", markersize=3)
    if plot_fit:
        plt.plot(x, curve, label="fit")
        plt.axvline(pi_x, ls="--", c="red", label=f"pi={pi_x:.2f}")
        plt.axvline(pi2_x, ls="--", c="red", label=f"pi/2={(pi2_x):.2f}")
    plt.legend(loc=4)
    plt.tight_layout()

    return pi_x, pi2_x

def convert2max_contrast(Is: np.ndarray, Qs: np.ndarray):
    """
    rotate the 2-d input data to maximize on the x-axis
    """

    # calculate the covariance matrix
    cov = np.cov(Is, Qs)

    # calculate the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(cov)

    # sort the eigenvectors by decreasing eigenvalues
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # rotate the data
    data = np.vstack([Is, Qs])
    data_rot = np.dot(eigenvectors.T, data)

    return data_rot[0], data_rot[1]


def T1_analyze(x: float, y: float, return_err=False, plot=True, max_contrast=False):
    if max_contrast:
        y, _ = convert2max_contrast(y.real, y.imag)
    else:
        y = np.abs(y)

    pOpt, pCov = ft.fitexp(x, y)
    y0, t1 = pOpt[0], pOpt[2]
    sim = ft.expfunc(x, *pOpt)
    err = np.sqrt(np.diag(pCov))

    if plot:
        t1_str = f"{t1:.2f}us +/- {err[2]:.2f}us"

        plt.figure()
        plt.plot(x, y, label="meas", ls="-", marker="o", markersize=3)
        plt.plot(x, sim, label="fit")
        plt.axhline(y0, ls="--", c="red", label=f"y0={y0:.3f}")
        plt.title(f"T1 = {t1_str}")
        plt.xlabel("Time (us)")
        plt.ylabel("Amplitude (a.u.)")
        plt.legend()
        plt.tight_layout()

    if return_err:
        return t1, err[2]
    return t1


def T2fringe_analyze(
    x: float, y: float, return_err=False, plot=True, max_contrast=False
):
    if max_contrast:
        y, _ = convert2max_contrast(y.real, y.imag)
    else:
        y = np.abs(y)

    pOpt, pCov = ft.fitdecaysin(x, y)
    y0, t2f, detune = pOpt[0], pOpt[4], pOpt[2]
    sim = ft.decaysin(x, *pOpt)
    err = np.sqrt(np.diag(pCov))

    if plot:
        t2f_str = f"{t2f:.2f}us +/- {err[4]:.2f}us"
        detune_str = f"{detune:.2f}MHz +/- {err[2]*1e3:.2f}kHz"

        plt.figure()
        plt.plot(x, y, label="meas", ls="-", marker="o", markersize=3)
        plt.plot(x, sim, label="fit")
        plt.axhline(y0, ls="--", c="red", label=f"y0={y0:.3f}")
        plt.title(f"T2 ramsey = {t2f_str}, detune = {detune_str}")
        plt.xlabel("Time (us)")
        plt.ylabel("Amplitude (a.u.)")
        plt.legend()
        plt.tight_layout()

    if return_err:
        return t2f, detune, err[4], err[2]
    return t2f, detune


def T2decay_analyze(
    x: float, y: float, return_err=False, plot=True, max_contrast=False
):
    if max_contrast:
        y, _ = convert2max_contrast(y.real, y.imag)
    else:
        y = np.abs(y)

    pOpt, pCov = ft.fitexp(x, y)
    y0, t2e = pOpt[0], pOpt[2]
    err = np.sqrt(np.diag(pCov))

    if plot:
        t2e_str = f"{t2e:.2f}us +/- {err[2]:.2f}us"

        plt.figure()
        plt.plot(x, y, label="meas", ls="-", marker="o", markersize=3)
        plt.plot(x, ft.expfunc(x, *pOpt), label="fit")
        plt.axhline(y0, ls="--", c="red", label=f"y0={y0:.3f}")
        plt.title(f"T2 echo = {t2e_str}")
        plt.xlabel("Time (us)")
        plt.ylabel("Amplitude (a.u.)")
        plt.legend()
        plt.tight_layout()

    if return_err:
        return t2e, err[2]
    return t2e

## Length Rabi

In [ ]:
# signals, Ts, _ = load_data("data/qub/len_rabi_q5_i_rf.hdf5")
signals, Ts, _ = load_data("data/qub/len_rabi_q5_i_fid.hdf5")
Ts = Ts * 1e6

rabi_analyze(Ts, signals, plot_fit=True, decay=True, max_contrast=False)
# plt.axhline(0.545, color='b', label="ground")
# plt.axhline(0.339, color='y', label='excited')
plt.axhline(2.055, color='b', label="ground")
plt.axhline(2.845, color='y', label="excited")
plt.xlabel('Time (us)')
plt.ylabel('Amplitude (a.u.)')
plt.show()

## Amp Rabi

In [ ]:
# signals, pdrs, _ = load_data("data/qub/amp_rabi_q5_i_rf.hdf5")
signals, pdrs, _ = load_data("data/qub/amp_rabi_q5_i_fid.hdf5")

rabi_analyze(pdrs, signals, plot_fit=True, decay=False, max_contrast=False)
# plt.axhline(0.545, color='b', label="ground")
# plt.axhline(0.339, color='y', label='excited')
plt.axhline(2.055, color='b', label="ground")
plt.axhline(2.845, color='y', label="excited")
plt.xlabel('Gain (a.u.)')
plt.ylabel('Amplitude (a.u.)')
plt.legend()
plt.show()

## T2 ramsey

In [ ]:
# signals, Ts, _ = load_data("data/qub/t2ramsey_q5_i_rf.hdf5")
signals, Ts, _ = load_data("data/qub/t2ramsey_q5_i_fid.hdf5")
Ts = Ts * 1e6

T2fringe_analyze(Ts, signals, plot=True, max_contrast=False)
# plt.axhline(0.545, color='b', label="ground")
# plt.axhline(0.339, color='y', label='excited')
plt.axhline(2.055, color='b', label="ground")
plt.axhline(2.845, color='y', label="excited")
plt.xlabel('Time (us)')
plt.ylabel('Amplitude (a.u.)')
plt.legend()
plt.show()

## T2 decay

In [ ]:
signals, Ts, _ = load_data("data/qub/t2decay_q5_i_rf.hdf5")
# signals, Ts, _ = load_data("data/qub/t2decay_q5_i_fid.hdf5")
Ts = Ts * 1e6

plt.figure()
T2decay_analyze(Ts, signals, plot=True, max_contrast=False)
plt.axhline(0.545, color='b', label="ground")
plt.axhline(0.339, color='y', label='excited')
# plt.axhline(2.055, color='b', label="ground")
# plt.axhline(2.845, color='y', label="excited")
plt.legend()
plt.show()

## T1

In [ ]:
# signals, Ts, _ = load_data("data/qub/t1_q5_i_rf.hdf5")
signals, Ts, _ = load_data("data/qub/t1_q5_i_fid.hdf5")
Ts = Ts * 1e6

T1_analyze(Ts, signals, plot=True, max_contrast=False)
# plt.axhline(0.545, color='b', label='ground')
# plt.axhline(0.339, color='y', label='excited')
plt.axhline(2.055, color='b', label="ground")
plt.axhline(2.845, color='y', label="excited")
plt.legend()
plt.show()

## T2 echo

In [ ]:
# signals, Ts, _ = load_data("data/qub/t2echo_q5_i_rf.hdf5")
signals, Ts, _ = load_data("data/qub/t2echo_q5_i_fid.hdf5")
Ts = Ts * 1e6

T2decay_analyze(Ts, signals, plot=True, max_contrast=False)
# plt.axhline(0.545, color='b', label="ground")
# plt.axhline(0.339, color='y', label='excited')
plt.axhline(2.055, color='b', label="ground")
plt.axhline(2.845, color='y', label="excited")
plt.legend()
plt.show()

## dispersive shift

In [10]:
def dispersive_analyze2(fpts, signals_g, signals_e):
    y_g = np.abs(signals_g)
    y_e = np.abs(signals_e)
    y_d = np.abs(signals_g - signals_e)

    # plot signals
    fig, ax = plt.subplots(2, 1)
    ax[0].plot(fpts, y_g, label="e", marker="o", markersize=3)
    ax[0].plot(fpts, y_e, label="g", marker="o", markersize=3)
    ax[0].legend()

    # plot difference and max/min points
    diff_curve = y_g - y_e
    max_fpt = fpts[np.argmax(diff_curve)]
    min_fpt = fpts[np.argmin(diff_curve)]
    abs_fpt = fpts[np.argmax(y_d)]
    ax[1].plot(fpts, diff_curve, label="abs", marker="o", markersize=3)
    ax[1].plot(fpts, y_d, label="iq", marker="o", markersize=3)
    ax[1].axvline(max_fpt, color="r", ls="--", label=f"max SNR1 = {max_fpt:.3f}")
    ax[1].axvline(min_fpt, color="g", ls="--", label=f"max SNR2 = {min_fpt:.3f}")
    ax[1].axvline(abs_fpt, color="b", ls="--", label=f"max IQ = {abs_fpt:.3f}")

    plt.legend(loc=2)
    plt.tight_layout()

    return fig, ax

In [ ]:
signals, fpts, _ = load_data("data/qub/disper_shift_q5_i_rf.hdf5")
# signals, fpts, _ = load_data("data/qub/disper_shift_q5_i_fid.hdf5")
fpts = fpts / 1e9
g_signals, e_signals = signals[:, 0], signals[:, 1]

fig, ax = dispersive_analyze2(fpts, g_signals, e_signals)
ax[0].set_ylabel("Amplitude (a.u.)")
ax[1].set_ylabel("Amplitude (a.u.)")
ax[1].set_xlabel("Frequency (GHz)")
plt.show()

# Single shot

In [13]:
# use plot in measure code